In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(0, None)
minVol = volRange[0]
maxVol = volRange[1]
cap = cv2.VideoCapture(0)
mp_hand = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
with mp_hand.Hands(min_detection_confidence=0.9, min_tracking_confidence=0.9, max_num_hands=2) as hands:
    pTime = 0
    while cap.isOpened():
        lmList = []
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(image, hand, mp_hand.HAND_CONNECTIONS, mp_draw.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), mp_draw.DrawingSpec(color=(0, 255, ), thickness=2, circle_radius=2))
            for id, lm in enumerate(results.multi_hand_landmarks[0].landmark):
                h,w,c=image.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                lmList.append([cx,cy])
            if len(lmList):
                x1, y1 = lmList[4][0], lmList[4][1]
                x2, y2 = lmList[8][0], lmList[8][1]
                cx, cy = (x1+x2)//2, (y1+y2)//2 
                cv2.circle(image,(x1,y1),10,(121, 22, 76),cv2.FILLED)
                cv2.circle(image,(x2,y2),10,(121, 22, 76),cv2.FILLED)
                cv2.line(image,(x1,y1),(x2,y2),(121, 22, 76),3,cv2.FILLED)
                cv2.circle(image, (cx, cy), 10, (121, 22, 76), cv2.FILLED)
                length = math.hypot(x1-x2, y1-y2)
                if length < 50:
                    cv2.circle(image, (cx, cy), 15, (0, 255, 0), cv2.FILLED)
                vol = np.interp(length, [50, 300], [minVol, maxVol])
                volBar = np.interp(length, [50, 300], [400, 150])
                volPer=np.interp(length, [50,300], [0,100])
                volume.SetMasterVolumeLevel(vol, None)
                cv2.rectangle(image,(50,150),(85,400),(0,255,0), 3, cv2.FILLED)
                cv2.rectangle(image,(50,int(volBar)),(85,400),(0,255,0),cv2.FILLED)
                cv2.putText(image, f'{int(volPer)}', (40, 450), cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1, (0, 255, 0), 2)
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
        cv2.imshow("Image", image)
        k = cv2.waitKey(1)
        if k == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()